In [1]:
!pip install semanticscholar

In [28]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [29]:
from semanticscholar import SemanticScholar

In [30]:
def s2_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Semantic scholar by keywords and return the results including abstracts.
    """
    from semanticscholar import SemanticScholar

    sch = SemanticScholar()
    
    search = sch.search_paper(query=query, limit = max_results)

    results = []
    for paper in search.items:
        try:
            results.append(
                {
                    "title": paper.title,
                    "authors": [author.name for author in paper.authors],
                    "published": paper.publicationDate.strftime("%Y-%m-%d"),
                    "abstract": paper.abstract,
                    "pdf_url": paper.openAccessPdf["url"]
                }
            )
        except:
            continue

    # # Write results to a file
    # with open('s2_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)

    return results

In [31]:


def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client(
        page_size=max_results,
        delay_seconds=5,
        num_retries=3
    )
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)

    return results

In [32]:
s2_search_tool = FunctionTool(
    s2_search, description="Search Semantic scholar for papers by keywords, returns found papers including abstracts"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)


In [33]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key = "<YOUR_API_KEY>"
)

s2_search_agent = AssistantAgent(
    name="Semantic_Scholar_Search_Agent",
    tools=[s2_search_tool],
    model_client=model_client,
    description="An agent that can search Semantic scholar paper database using keywords related to given topic",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=model_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=model_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

In [34]:
termination = TextMentionTermination("TERMINATE")
team = MagenticOneGroupChat(
    participants=[s2_search_agent, arxiv_search_agent, report_agent], 
    termination_condition=termination,
    model_client = model_client
)

In [37]:
await Console(
    team.run_stream(
        task="Make a search of 20 recent publications and write a literature review on recent advances in deep learning optimizers with an accent on novel optimization algorythms",
    )
);

#await model_client.close()

---------- TextMessage (user) ----------
Make a search of 20 recent publications and write a literature review on recent advances in deep learning optimizers with an accent on novel optimization algorythms
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Make a search of 20 recent publications and write a literature review on recent advances in deep learning optimizers with an accent on novel optimization algorythms


To answer this request we have assembled the following team:

Semantic_Scholar_Search_Agent: An agent that can search Semantic scholar paper database using keywords related to given topic
Arxiv_Search_Agent: An agent that can search Arxiv for papers related to a given topic, including abstracts
Report_Agent: Generate a report based on a given topic


Here is an initial fact sheet to consider:

1. **GIVEN OR VERIFIED FACTS**
   - The request involves a literature review.
   - The focus is on recent advances

In [ ]:
team.

# Literature Review on Recent Advances in Machine Learning Optimizers

The field of optimization in machine learning has seen significant advancements in recent years. These developments include novel algorithms, techniques for integrating machine learning with traditional optimization methods, and applications across various domains. This literature review synthesizes the key findings from recent studies highlighting innovative approaches to optimization problems in machine learning.

## 1. Novel Optimization Techniques

One notable advancement is the integration of Optimal Transport (OT) theory into machine learning optimization methodologies. OT provides a probabilistic framework for comparing probability distributions, which has proven useful for data-driven tasks such as generative modeling and transfer learning. Montesuma et al. (2023) discussed significant contributions in this area, emphasizing recent innovations in computational OT methods and their implications for various aspects of machine learning, including supervised and unsupervised learning paradigms (Montesuma, E. F., Mboula, F. N., & Souloumiac, A. (2023). *Recent Advances in Optimal Transport for Machine Learning*. arXiv:2306.16156).

Further, recent studies have harnessed machine learning to enhance conventional optimization techniques, specifically in areas like model predictive control (MPC). E et al. (2022) highlighted how machine learning can empower control solvers, addressing challenges inherent in tackling complex optimal control problems (E, W., Han, J., & Long, J. (2022). *Empowering Optimal Control with Machine Learning: A Perspective from Model Predictive Control*. arXiv:2205.07990).

## 2. Constrained Optimization

The challenge of integrating machine learning with combinatorial optimization is another area garnering attention. Kotary et al. (2021) provided a comprehensive survey on leveraging machine learning to solve constrained optimization problems. They emphasized the potential for hybrid models that can generate fast, approximate solutions through the combination of machine learning architectures and combinatorial optimizers (Kotary, J., Fioretto, F., Van Hentenryck, P., & Wilder, B. (2021). *End-to-End Constrained Optimization Learning: A Survey*. arXiv:2103.16378).

## 3. Process Optimization

The application of machine learning in process optimization, especially within chemical engineering, has also seen notable advances. Mitrai and Daoutidis (2024) reviewed strategies for automating the selection and tuning of optimization algorithms based on learned behaviors from numerical solvers. They discussed the importance of representing decision-making problems effectively for machine learning tasks (Mitrai, I., & Daoutidis, P. (2024). *Accelerating Process Control and Optimization via Machine Learning: A Review*. arXiv:2412.18529).

## 4. The Interplay Between Machine Learning and Mathematical Programming

The integration of machine learning and mathematical programming has opened up new avenues for optimization under uncertainty. Ning and You (2019) explored how data-driven optimization can synergistically link machine learning with traditional optimization frameworks, facilitating better decision-making under uncertain conditions. Their review covered various data-driven approaches, identifying critical research opportunities in this emerging inter-disciplinary field (Ning, C., & You, F. (2019). *Optimization under Uncertainty in the Era of Big Data and Deep Learning: When Machine Learning Meets Mathematical Programming*. arXiv:1904.01934).

## 5. Hybrid Models and Future Directions

The emergent trend toward hybrid models that combine classical optimization techniques with machine learning is becoming increasingly prominent. These models have demonstrated their capability in tackling intricate systems where conventional methods fall short. As research continues to evolve, scholars advocate for more adaptive algorithms that can learn from their performance dynamically, which has significant implications for real-time optimization scenarios.

## Conclusion

In conclusion, recent advancements in machine learning optimizers reveal a clear trajectory toward the integration of traditional optimization strategies with machine learning innovations. This synthesis of techniques not only enhances the efficiency of optimization processes but also broadens their applicability across diverse fields. Future research is expected to delve deeper into hybrid models, the interactions between learning and optimization, and uncertainty handling mechanisms in complex environments.

### References

E, W., Han, J., & Long, J. (2022). Empowering Optimal Control with Machine Learning: A Perspective from Model Predictive Control. *arXiv:2205.07990*.  

Mitrai, I., & Daoutidis, P. (2024). Accelerating Process Control and Optimization via Machine Learning: A Review. *arXiv:2412.18529*.  

Montesuma, E. F., Mboula, F. N., & Souloumiac, A. (2023). Recent Advances in Optimal Transport for Machine Learning. *arXiv:2306.16156*.  

Ning, C., & You, F. (2019). Optimization under Uncertainty in the Era of Big Data and Deep Learning: When Machine Learning Meets Mathematical Programming. *arXiv:1904.01934*.

Kotary, J., Fioretto, F., Van Hentenryck, P., & Wilder, B. (2021). End-to-End Constrained Optimization Learning: A Survey. *arXiv:2103.16378*.


In [38]:
import requests

response = requests.get(
    "https://api.aimlapi.com/models",
    headers={"Accept":"*/*"},
)

data = response.json()

In [66]:
import json
import pandas as pd
df_models = pd.DataFrame(json.loads(response.content)["data"])

In [67]:
df_models = pd.concat([df_models,pd.json_normalize(df_models["info"])], axis = 1).drop("info", axis = "columns")

In [82]:
df_models[380:420]

,id,type,features,endpoints,name,developer,description,contextLength,maxTokens,url,docs_url
380,stable-audio,audio,[],[/v2/generate/audio],Stable audio,Stability AI,Stable Audio generates high-quality audio from...,NaN,NaN,https://aimlapi.com/models/stable-audio,https://docs.aimlapi.com/api-references/music-...
381,minimax-music,audio,[],[/v2/generate/audio],Minimax music,Minimax AI,,NaN,NaN,,https://docs.aimlapi.com/api-references/music-...
382,google/lyria2,audio,[],[/v2/generate/audio],Lyria 2,Google,,NaN,NaN,,https://docs.aimlapi.com/api-references/music-...
383,music-01,audio,[],"[v2/generate/audio/minimax/upload, v2/generate...",Music 01,Minimax AI,MiniMax Music generates high-quality music fro...,NaN,NaN,https://aimlapi.com/models/minimax-music-api,https://docs.aimlapi.com/api-references/music-...
384,minimax/music-1.5,audio,[],[v2/generate/audio/minimax/generate],MiniMax Music 1.5,Minimax AI,MiniMax Music 1.5 provides enhanced music gene...,NaN,NaN,,
385,minimax/music-2.0,audio,[],[v2/generate/audio/minimax/generate],MiniMax Music 2.0,Minimax AI,MiniMax Music 2.0 delivers superior music gene...,NaN,NaN,,
386,elevenlabs/eleven_music,audio,[],[/v2/generate/audio],Eleven Music v1,Elevenlabs,,NaN,NaN,,https://docs.aimlapi.com/api-references/music-...


In [78]:
df_models.iloc[220]["description"]

'Aura is a real-time TTS model with human-like voices for conversational AI applications.'